In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

# Read model (model of Escherichia coli)
model = read_sbml_model('iML1515.xml')

In [31]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.004565,0,0.00%
cl_e,EX_cl_e,0.004565,0,0.00%
cobalt2_e,EX_cobalt2_e,2.192E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006218,0,0.00%
fe2_e,EX_fe2_e,0.01409,0,0.00%
glc__D_e,EX_glc__D_e,10,6,100.00%
k_e,EX_k_e,0.1712,0,0.00%
mg2_e,EX_mg2_e,0.007608,0,0.00%
mn2_e,EX_mn2_e,0.000606,0,0.00%
mobd_e,EX_mobd_e,6.139E-06,0,0.00%


In [6]:
model.metabolites.get_by_id('34hpp_c')
#starting point
#add 5 reactions: 
#homogentisate, 
#2-Methyl-6-phytylquinol, 
#2,3-Dimethyl-5-phytylquinol, 
#gamma-Tocopherol,
#alpha-Tocopherol

Metabolite identifier,34hpp_c
Name,3-(4-Hydroxyphenyl)pyruvate
Memory address,0x7fb2d720c760
Formula,C9H7O4
Compartment,c
In 2 reaction(s),"PPND, TYRTA"


In [7]:
# Reaction 1: 3-(4-Hydroxyphenyl)pyruvate + O2 O2 --> CO2 CO2 + Homogentisate
new_reaction1 = Reaction('34HPPOR') # 4 Hydroxyphenylpyruvateoxygen oxidoreductase
homogentisate = Metabolite(id='hgentis_c', compartment='c')
new_reaction1.add_metabolites({model.metabolites.get_by_id('34hpp_c'): -1, # 3-(4-Hydroxyphenyl)pyruvate
                               model.metabolites.o2_c: -1, #O2
                               homogentisate: 1, # Homogentisate
                               model.metabolites.co2_c: 1, # CO2
                              })
model.add_reactions([new_reaction1])

In [8]:
# Reaction 2: h_c + hgentis_c + phdp_c ⇌ co2_c + ppi_c + 2m6phol_c
new_reaction2 = Reaction('HGPHT') # Homogenitisate phytyltransferase
methylphytylquinol = Metabolite(id='2m6phol_c', compartment='c')
phytyldiphosphate = Metabolite(id='phdp_c', compartment='c') # Phytyl diphosphate
new_reaction2.add_metabolites({model.metabolites.hgentis_c: -1, # Homogentisate
                               model.metabolites.h_c: -1, # H+
                               phytyldiphosphate: -1,
                               methylphytylquinol: 1, # 2-Methyl-6-phytylquinol
                               model.metabolites.ppi_c: 1, # diphosphate
                               model.metabolites.co2_c: 1, # CO2
                              })
model.add_reactions([new_reaction2])

In [9]:
# Reaction 3: amet_c + 2m6phol_c ⇌ ahcys_c + h_c + 23dmphol_c
new_reaction3 = Reaction('MPBQ') # 2-methyl-6-phytylbenzoquinone methyltransferase
dimethylphytylquinol = Metabolite(id='23dmphol_c', compartment='c')
new_reaction3.add_metabolites({model.metabolites.get_by_id('2m6phol_c'): -1, # 2-Methyl-6-phytylquinol
                               model.metabolites.amet_c: -1, # S-Adenosyl-L-methionine
                               dimethylphytylquinol: 1, # 2,3-Dimethyl-5-phytylquinol
                               model.metabolites.ahcys_c: 1, # S-Adenosyl-L-homocysteine
                               model.metabolites.h_c: 1, # H+
                              })
model.add_reactions([new_reaction3])

In [10]:
# Reaction 4: 23dmphol_c ⇌ gtocophe_c
new_reaction4 = Reaction('TOCOPHS1') # Tocopherol cyclase (g)
gammatocopherol = Metabolite(id='gtocophe_c', compartment='c')
new_reaction4.add_metabolites({model.metabolites.get_by_id('23dmphol_c'): -1, # 2,3-Dimethyl-5-phytylquinol
                               gammatocopherol: 1, # Gamma-Tocopherol
                              })
model.add_reactions([new_reaction4])

In [11]:
# Reaction 5: amet_c + gtocophe_c ⇌ ahcys_c + h_c + avite1_c
new_reaction5 = Reaction('TOCOPHOM1') # Tocopherol O-methyltransferase
alphatocopherol = Metabolite(id='avite1_c', compartment='c')
new_reaction5.add_metabolites({model.metabolites.gtocophe_c: -1, # Gamma-Tocopherol
                               model.metabolites.amet_c: -1, # S-Adenosyl-L-methionine
                               alphatocopherol: 1, # Alpha-Tocopherol
                               model.metabolites.ahcys_c: 1, # S-Adenosyl-L-homocysteine
                               model.metabolites.h_c: 1, # H+
                              })
model.add_reactions([new_reaction5])

In [12]:
# Co-factor production
# Reaction 6: frdp_c + ipdp_c ⇌ ggdp_c + h_c + ppi_c
new_reaction6 = Reaction('GGPS') # Geranylgeranyl diphosphate synthase
geranylgeranyldiphosphate = Metabolite(id='ggdp_c', compartment='c')
new_reaction6.add_metabolites({model.metabolites.frdp_c: -1, # Farnesyl diphosphate
                               model.metabolites.ipdp_c: -1, # Isopentenyl diphosphat
                               geranylgeranyldiphosphate: 1, # Geranylgeranyl diphosphate
                               model.metabolites.ppi_c: 1, # diphosphate
                               model.metabolites.h_c: 1, # H+
                              })
model.add_reactions([new_reaction6])

In [13]:
# Co-factor production
# Reaction 7: ggdp_c + 3.0 h_c + 3.0 nadph_c ⇌ 3.0 nadp_c + phdp_c
new_reaction7 = Reaction('GGDR') # Geranylgeranyl diphosphate reductase
new_reaction7.add_metabolites({model.metabolites.ggdp_c: -1, # Geranylgeranyl diphosphate
                               model.metabolites.h_c: -3, # H+
                               model.metabolites.nadph_c: -3, # NADPH
                               model.metabolites.nadp_c: 3, # NADP
                               model.metabolites.phdp_c: 1, # # Phytyl diphosphate
                              })
model.add_reactions([new_reaction7])

In [14]:
#production of vitamin E (avite1_c)
model.add_boundary(model.metabolites.avite1_c, type='demand')

with model:
    model.objective = model.reactions.TOCOPHOM1
    lets_produce_avite1 = model.optimize().objective_value

In [17]:
#production rate of vitamin E µmax
lets_produce_avite1

1.222850031113874

In [30]:
from cobra.io import write_sbml_model
write_sbml_model(model,"model_atocopherol.xml")

# Alternative Pathway via delta-tocopherol

In [2]:
new_model = read_sbml_model('iML1515.xml')

In [3]:
# Reaction 1: 3-(4-Hydroxyphenyl)pyruvate + O2 O2 --> CO2 CO2 + Homogentisate
new_reaction1 = Reaction('34HPPOR') # 4 Hydroxyphenylpyruvateoxygen oxidoreductase
homogentisate = Metabolite(id='hgentis_c', compartment='c')
new_reaction1.add_metabolites({new_model.metabolites.get_by_id('34hpp_c'): -1, # 3-(4-Hydroxyphenyl)pyruvate
                               new_model.metabolites.o2_c: -1, #O2
                               homogentisate: 1, # Homogentisate
                               new_model.metabolites.co2_c: 1, # CO2
                              })
new_model.add_reactions([new_reaction1])

In [4]:
# Reaction 2: h_c + hgentis_c + phdp_c ⇌ co2_c + ppi_c + 2m6phol_c
new_reaction2 = Reaction('HGPHT') # Homogenitisate phytyltransferase
methylphytylquinol = Metabolite(id='2m6phol_c', compartment='c')
phytyldiphosphate = Metabolite(id='phdp_c', compartment='c') # Phytyl diphosphate
new_reaction2.add_metabolites({new_model.metabolites.hgentis_c: -1, # Homogentisate
                               new_model.metabolites.h_c: -1, # H+
                               phytyldiphosphate: -1,
                               methylphytylquinol: 1, # 2-Methyl-6-phytylquinol
                               new_model.metabolites.ppi_c: 1, # diphosphate
                               new_model.metabolites.co2_c: 1, # CO2
                              })
new_model.add_reactions([new_reaction2])

In [5]:
# Reaction 3: 2m6phol_c ⇌ dtocophe_c
new_reaction3 = Reaction('TCYD') # Tocopherol cyclase (delta)
deltatocopherol = Metabolite(id='dtocophe_c', compartment='c') # delta-tocopherol
new_reaction3.add_metabolites({new_model.metabolites.get_by_id('2m6phol_c'): -1, #  2-Methyl-6-phytylquinol
                               deltatocopherol: 1, # delta-tocopherol
                              })
new_model.add_reactions([new_reaction3])

In [6]:
# Reaction 4: amet_c + dtocophe_c ⇌ ahcys_c + h_c + bvite_c
new_reaction4 = Reaction('TOCOPHOM2') # Tocopherol O-methyltransferase(2)
betatocopherol = Metabolite(id='bvite_c', compartment='c') # beta-tocopherol
new_reaction4.add_metabolites({new_model.metabolites.dtocophe_c: -1, # delta-Tocopherol
                               new_model.metabolites.amet_c: -1, # S-Adenosyl-L-methionine
                               betatocopherol: 1, # beta-Tocopherol
                               new_model.metabolites.ahcys_c: 1, # S-Adenosyl-L-homocysteine
                               new_model.metabolites.h_c: 1, # H+
                               })
new_model.add_reactions([new_reaction4])

In [7]:
# Reaction 5: bvite_c ⇌ avite1_c
new_reaction5 = Reaction('R07508') #
alphatocopherol = Metabolite(id='avite1_c', compartment='c') # alpha-tocopherol
new_reaction5.add_metabolites({new_model.metabolites.bvite_c: -1, #  beta-tocopherol
                               alphatocopherol: 1, # alpha-tocopherol
                              })
new_model.add_reactions([new_reaction5])

In [8]:
# Co-factor production
# Reaction 6: frdp_c + ipdp_c ⇌ ggdp_c + h_c + ppi_c
new_reaction6 = Reaction('GGPS') # Geranylgeranyl diphosphate synthase
geranylgeranyldiphosphate = Metabolite(id='ggdp_c', compartment='c')
new_reaction6.add_metabolites({new_model.metabolites.frdp_c: -1, # Farnesyl diphosphate
                               new_model.metabolites.ipdp_c: -1, # Isopentenyl diphosphat
                               geranylgeranyldiphosphate: 1, # Geranylgeranyl diphosphate
                               new_model.metabolites.ppi_c: 1, # diphosphate
                               new_model.metabolites.h_c: 1, # H+
                              })
new_model.add_reactions([new_reaction6])

In [9]:
# Co-factor production
# Reaction 7: ggdp_c + 3.0 h_c + 3.0 nadph_c ⇌ 3.0 nadp_c + phdp_c
new_reaction7 = Reaction('GGDR') # Geranylgeranyl diphosphate reductase
new_reaction7.add_metabolites({new_model.metabolites.ggdp_c: -1, # Geranylgeranyl diphosphate
                               new_model.metabolites.h_c: -3, # H+
                               new_model.metabolites.nadph_c: -3, # NADPH
                               new_model.metabolites.nadp_c: 3, # NADP
                               new_model.metabolites.phdp_c: 1, # # Phytyl diphosphate
                              })
new_model.add_reactions([new_reaction7])

In [10]:
#production of vitamin E (avite1_c)
new_model.add_boundary(new_model.metabolites.avite1_c, type='demand')

with new_model:
    new_model.objective = new_model.reactions.R07508
    lets_produce_avite1 = new_model.optimize().objective_value

In [11]:
lets_produce_avite1

1.2966809633784173

The production rate of alpha-tocopherol in the second pathway is slightly higher than in the first pathway. The number of added reactions is the same for both pathways (5 reactions + 2 reactions for co-factor production).
However, the enzyme that catalyzes the reaction of beta-tocopherol to alpha-tocopherol is not yet characterized so the addition of this step is only theoretical.